# This is Unsupervised machine learning and clustering problem so, We will be using Collaborative filtering rather than Content Based Filtering or Hybrid filtering

In [1]:
import pandas as pd
import numpy as np

### Downloaded the datasets from IIF(Institute Of Informatik FreiBurg)

## First dataset

In [2]:
books = pd.read_csv("BX-Books.csv", sep=';', error_bad_lines=False, encoding='latin-1')

C:\Users\miftahul hussain\AppData\Local\Temp\ipykernel_21384\3738883866.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv("BX-Books.csv", sep=';', error_bad_lines=False, encoding='latin-1')
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping l

In [3]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [4]:
books.shape

(271360, 8)

In [5]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

# Removing unnecessary columns

In [6]:
books = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


## Renaming columns

In [7]:
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'},inplace=True)
books.head(2)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


# Second dataset

In [8]:
users = pd.read_csv('BX-Users.csv',sep=';',error_bad_lines=False, encoding='latin-1')

C:\Users\miftahul hussain\AppData\Local\Temp\ipykernel_21384\3503304191.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv('BX-Users.csv',sep=';',error_bad_lines=False, encoding='latin-1')


In [9]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [10]:
users.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)
users.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [11]:
users.shape

(278858, 3)

# Third dataset

In [12]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';',error_bad_lines=False, encoding='latin-1')

C:\Users\miftahul hussain\AppData\Local\Temp\ipykernel_21384\1380700623.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';',error_bad_lines=False, encoding='latin-1')


In [13]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [14]:
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [15]:
ratings.shape

(1149780, 3)

## For collaborative filtering, we will be creating a matrix where we will consider only those users whose rating > 200 and the books whose ratings >= 50

In [16]:
x = ratings['user_id'].value_counts() > 200
x

11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: user_id, Length: 105283, dtype: bool

In [17]:
x[x].shape

(899,)

## So there are 899 user whose rating > 200 and we will be using these users for book recommendations

In [18]:
y = x[x].index
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [19]:
ratings = ratings[ratings['user_id'].isin(y)]

In [20]:
ratings.shape

(526356, 3)

## ratings rows has been reduced from 11.4 lakh to 5.2 lakh

# Merging Ratings with books

In [21]:
ratings_with_books = ratings.merge(books, on='ISBN')

In [22]:
ratings_with_books.head(5)

,user_id,ISBN,rating,title,author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [23]:
ratings_with_books.shape

(487671, 7)

In [54]:
ratings_with_books['rating'].value_counts()

0     366409
8      28529
10     26268
9      21576
7      18611
5      14695
6       7968
4       1577
3       1020
2        556
1        462
Name: rating, dtype: int64

## Ratings rows hasbeen reduced from 5.2 l to 4.8 l beacuse some of the books have no ISBN i.e data missing so they have been removed

## The  books that got ratings >= 50

In [25]:
number_rating = ratings_with_books.groupby(['title'])['rating'].count().reset_index()

In [58]:
number_rating.rename(columns={'rating':'number of ratings'},inplace=True)
number_rating.sample(3)

,title,number of ratings
99651,Rubyfruit Jungle,35
42081,Families and How to Survive Them,1
150408,Untamed (Mira),1


In [27]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [75]:
final_rating.sample(4)

,user_id,ISBN,rating,title,author,year,publisher,number of ratings
18953,101851,0440211727,9,A Time to Kill,JOHN GRISHAM,1992,Dell,210
26169,238864,034538475X,0,The Tale of the Body Thief (Vampire Chronicles...,Anne Rice,1993,Ballantine Books,81
3257,85993,0345452534,0,The Murder Book,JONATHAN KELLERMAN,2002,Ballantine Books,86
136488,43842,0425099334,0,Shattered,Dean R. Koontz,1993,Berkley Publishing Group,58


In [29]:
final_rating.shape

(487671, 8)

In [30]:
final_rating = final_rating[final_rating['number of ratings'] >= 50]

In [78]:
final_rating.sample(5)

,user_id,ISBN,rating,title,author,year,publisher,number of ratings
30373,254,0380789035,10,American Gods,Neil Gaiman,2002,HarperTorch,94
133048,185233,0425178765,0,Easy Prey,John Sandford,2001,Berkley Publishing Group,73
93766,11676,0679745203,10,The English Patient,Michael Ondaatje,1996,Vintage Books USA,93
89920,95903,0767914767,10,The Devil Wears Prada : A Novel,LAUREN WEISBERGER,2004,Broadway Books,75
62311,40943,0805063897,0,Nickel and Dimed: On (Not) Getting By in America,Barbara Ehrenreich,2002,Owl Books,112


In [32]:
final_rating.shape

(61853, 8)

## so these are the users and books ratings whose rating > 200 and the books whose ratings >= 50

In [33]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [34]:
final_rating.shape

(59850, 8)

In [35]:
book_pivot = final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [36]:
book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN


In [37]:
book_pivot.shape

(742, 888)

In [38]:
book_pivot.fillna(0,inplace=True)

In [39]:
book_pivot.head(3)

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## We will convert this book_pivot into a sparse matrix so that all the zeroes would be ommited and only the values would be considered

In [40]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [41]:
type(book_sparse)

scipy.sparse.csr.csr_matrix

# Model Building

## We will be using Nearest Neighbors algo since it is a clustering problem and not a classification problem like knn

In [42]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')  # brute means sabkasabke saath distance nikalo

In [43]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

## Calculating the distances of the nearest five books and getting their suggestions

In [44]:
distances,suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1),n_neighbors=6)

In [45]:
distances  # so these are the 5 nearest distance books to 0

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [46]:
suggestions  # These are the user_id of the books

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [47]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

## Providing the user_id and getting the recommendations

In [48]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [49]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [50]:
np.where(book_pivot.index == 'Animal Farm')[0][0]

54

## Providing the title of books and getting the recommended books

In [51]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distances,suggestions = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1),n_neighbors=6)
    
    for i in range(len(suggestions)):
        if i==0:
            print("The suggestions for", book_name, "are: ")
        
        if not i:  
            print(book_pivot.index[suggestions[i]])

In [52]:
recommend_book('Harry Potter and the Chamber of Secrets (Book 2)')

The suggestions for Harry Potter and the Chamber of Secrets (Book 2) are: 
Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


# Our Book_Recommender_system project is complete. By Giving any book name to the function it will recommend you the related books